---
title: Mini Dashboard
format: dashboard
---


## Row {height=70%}


In [ ]:
#| echo: false

# Load data directly (instead of from __quarto__)
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geo_utils import load_shapefile_from_zip
from ipyleaflet import Map, Marker, MarkerCluster, GeoJSON, LayersControl, basemap_to_tiles, basemaps
import ipywidgets as widgets

# Load Fan boundary
zip_path = "../data/Neighborhoods-shp.zip"
gdf_neigh = load_shapefile_from_zip(zip_path).to_crs("EPSG:4326")
fan_gdf = gdf_neigh[gdf_neigh["Name"].str.strip().str.lower() == "the fan"].copy()
fan_gdf["geometry"] = fan_gdf.geometry.simplify(0.0001, preserve_topology=True)
fan_boundary = fan_gdf.unary_union

# Load geocoded homes
df_hht = pd.read_csv("../data/geocoded_hht_homes.csv")
gdf_hht = gpd.GeoDataFrame(
    df_hht,
    geometry=gpd.points_from_xy(df_hht.lon, df_hht.lat),
    crs="EPSG:4326"
)
homes_in_fan = gdf_hht[gdf_hht.geometry.within(fan_boundary)].copy()

# Base map
m = Map(
    center=(37.5525, -77.465556),
    zoom=15,
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
)

# Fan boundary layer
m.add_layer(GeoJSON(
    data=fan_gdf.__geo_interface__,
    style={"color": "blue", "fillOpacity": 0.05, "weight": 2},
    name="Fan Boundary"
))

# Add all homes
markers = [
    Marker(location=(r.geometry.y, r.geometry.x),
           title=f"{r['Address']} ({r['Year']})")
    for _, r in homes_in_fan.iterrows()
]
m.add_layer(MarkerCluster(markers=markers, name="All Homes"))
m.add_control(LayersControl(position="topright"))

# Search bar
search = widgets.Text(placeholder="Filter by address, owner, or year…", description="Search:")
display(search)

def update_map(change):
    term = change["new"]
    filt = homes_in_fan[
        homes_in_fan["Address"].str.contains(term, case=False, na=False) |
        homes_in_fan["Owner"].str.contains(term, case=False, na=False) |
        homes_in_fan["Year"].astype(str).str.contains(term)
    ]
    m.layers = m.layers[:2]  # Keep base + boundary
    new_markers = [
        Marker(location=(r.geometry.y, r.geometry.x),
               title=f"{r['Address']} ({r['Year']})")
        for _, r in filt.iterrows()
    ]
    m.add_layer(MarkerCluster(markers=new_markers, name="Filtered Homes"))

search.observe(update_map, names="value")
m

## Row {height=15%}


In [ ]:
#| echo: false

homes = homes_in_fan.copy()
homes["Street"] = homes["Address"].str.extract(r"\d+\s+(.*)")[0]
sorted_df = homes.sort_values("Street")
sorted_df[["Address", "Owner", "Year"]].reset_index(drop=True).head(20)

## Row {height=15%}

In [ ]:
#| layout-ncol: 4
#| echo: false

homes = homes_in_fan
num_years     = homes["Year"].nunique()
num_homes     = homes["Address"].nunique()
repeat_homes  = (homes["Address"].value_counts() > 1).sum()
unique_owners = homes["Owner"].nunique()

[
  {"title":"Years",        "value": num_years,     "icon":"calendar", "color":"primary"},
  {"title":"Unique Homes", "value": num_homes,     "icon":"home",     "color":"primary"},
  {"title":"Repeat Homes", "value": repeat_homes,  "icon":"refresh",  "color":"primary"},
  {"title":"Owners",       "value": unique_owners, "icon":"users",    "color":"primary"}
]